In [ ]:
#installing essential libraries
!pip install mido
!pip install np_utils
!pip install music21

In [47]:
#importing all necessary libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from music21 import *
from mido import MidiFile
from keras.utils.np_utils import to_categorical
import os
import numpy 
import pandas 
import glob
import pickle


**DATA LOADING AND PREPROCESSING**

In [ ]:
#Loading the list of mozart midi files as stream 
filepath = "../input/classical-music-midi/mozart/"
#Getting midi files
all_midis= []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath+i
        midi = converter.parse(tr)
        all_midis.append(midi)

In [ ]:
#Extracting the notes from the files to be used as an training data       
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    #appending the notes
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    #appending all the notes in Chord
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as DarkNotes
DarkNotes = extract_notes(all_midis)
print("Total notes in all the Mozart midis in the dataset:", len(DarkNotes))

In [88]:
#Preparing the sequence to be fed in Neural network
def prepare_sequences(notes, n_vocab):
    sequence_length = 100  # Taking 100 past values to predict 

    # get all pitch names and create a map from notes to integers
    unique_pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(unique_pitchnames))
   
    network_input = []
    network_output = []
    # create input sequences and the corresponding outputs according to sequence_length
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
        
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    # One hot encoding for categorical data
    network_output = to_categorical(network_output)
    return (network_input, network_output)



**MODEL ARCHITECTURE**

In [89]:
#creating the model architecture
def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [92]:
# Saving the weights for future use 
def train(model, network_input, network_output):
    filepath = "weights-improvement-{epoch:02d}-bigger.hdf5"
    batch_size=128
    checkpoint = ModelCheckpoint(
        filepath=filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min',
        period= 10
    )
    callbacks_list = [checkpoint]
    model.fit(network_input, network_output, epochs=100, batch_size=batch_size, callbacks=callbacks_list)
    


**TRANING THE NEURAL NETWORK**

In [93]:
#Train the neural network to generate Music
def train_network():
    notes = DarkNotes
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)
    model = create_network(network_input, n_vocab)
    train(model, network_input, network_output)

train_network()

12024
Epoch 1/100
94/94 [==============================] - 74s 724ms/step - loss: 5.3891
Epoch 2/100
94/94 [==============================] - 69s 723ms/step - loss: 5.1094
Epoch 3/100
94/94 [==============================] - 68s 725ms/step - loss: 4.9140
Epoch 4/100
94/94 [==============================] - 68s 726ms/step - loss: 4.7750
Epoch 5/100
94/94 [==============================] - 70s 737ms/step - loss: 4.7023
Epoch 6/100
94/94 [==============================] - 68s 719ms/step - loss: 4.6603
Epoch 7/100
94/94 [==============================] - 68s 724ms/step - loss: 4.6225
Epoch 8/100
94/94 [==============================] - 69s 737ms/step - loss: 4.5865
Epoch 9/100
94/94 [==============================] - 68s 726ms/step - loss: 4.5616
Epoch 10/100
94/94 [==============================] - 67s 716ms/step - loss: 4.5228
Epoch 11/100
94/94 [==============================] - 67s 718ms/step - loss: 4.5021
Epoch 12/100
94/94 [==============================] - 69s 730ms/step - loss: 4.

**GENERATE MUSIC NOTES**

In [154]:
def generate():
    notes = DarkNotes
    n_vocab = len(set(notes))
    pitchnames = sorted(set(item for item in notes))
    network_input, normalized_input = prepare_sequences_for_generation(notes,pitchnames, n_vocab)
    
    model = create_network_for_generation(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

**HELPER FUNCTIONS**

In [155]:
# Prepare the sequences used by neural networks

def prepare_sequences_for_generation(notes, pitchnames, n_vocab):
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)
    
    return (network_input, normalized_input)

In [156]:
# create the neural network and load weights
def create_network_for_generation(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('/kaggle/working/weights-improvement-100-bigger.hdf5')

    return model

In [157]:
#Generate notes from the neural network based on a sequence of notes
def generate_notes(model, network_input, pitchnames, n_vocab):
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    # reverse mapping to get notes from int
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(600):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    
    return prediction_output


In [158]:
#convert the output from the prediction to notes and create a midi file from the notes
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [159]:
generate()